In [1]:
import tensorflow as tf
import string
import requests

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
file_path = "/kaggle/input/shakespeare1-txt/shakespeare1.txt"
with open(file_path, "r", encoding="utf-8") as file1:
    response = file1.read()

In [3]:
response[0]

'T'

In [4]:
data = response.split('\n')
data[0]

'This is the 100th Etext file presented by Project Gutenberg, and'

In [5]:
data = data[253:]

In [6]:
data[0]

'  From fairest creatures we desire increase,'

In [7]:
len(data)

124204

In [8]:
data = " ".join(data)

In [9]:
data[100]

't'

In [10]:
def cleantext(doc):
  tokens = doc.split()
  table = str.maketrans('','',string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower() for word in tokens]
  return tokens

In [11]:
tokens = cleantext(data)
print(tokens[:50])

['from', 'fairest', 'creatures', 'we', 'desire', 'increase', 'that', 'thereby', 'beautys', 'rose', 'might', 'never', 'die', 'but', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease', 'his', 'tender', 'heir', 'might', 'bear', 'his', 'memory', 'but', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes', 'feedst', 'thy', 'lights', 'flame', 'with', 'selfsubstantial', 'fuel', 'making', 'a', 'famine', 'where', 'abundance', 'lies', 'thy']


In [12]:
len(tokens)

898199

In [13]:
len(set(tokens)) #unique words

27956

In [14]:
length = 50 + 1
lines = []

for i in range(length, len(tokens)):
  seq = tokens[i - length:i]
  line = ' '.join(seq)
  lines.append(line)
  if i > 200000:
    break

print(len(lines))

199951


In [15]:
lines[0]

'from fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self'

In [16]:
tokens[50]

'self'

In [17]:
lines[1]

'fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self thy'

## Build LSTM Model and Prepare X & Y

In [18]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [19]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [20]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]

In [21]:
X[0]

array([   47,  1408,  1264,    37,   451,  1406,     9,  2766,  1158,
        1213,   171,   132,   269,    20,    24,     1,  4782,    87,
          30,    98,  4781,    18,   715,  1263,   171,   211,    18,
         829,    20,    27,  3807,     4,   214,   121,  1212,   153,
       13004,    31,  2765,  1847,    16, 13003, 13002,   754,     7,
        3806,    99,  2430,   466,    31])

In [22]:
y[0]

307

In [23]:
vocab_size = len(tokenizer.word_index) + 1

In [24]:
y = to_categorical(y, num_classes = vocab_size)
X.shape[1]

50

In [25]:
seq_length = X.shape[1]

## LSTM Model

In [26]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length = seq_length))
model.add(LSTM(50, return_sequences = True))
model.add(LSTM(100))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(vocab_size, activation = 'softmax'))

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            650450    
                                                                 
 lstm (LSTM)                 (None, 50, 50)            20200     
                                                                 
 lstm_1 (LSTM)               (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 512)               51712     
                                                                 
 dense_1 (Dense)             (None, 13009)             6673617   
                                                                 
Total params: 7456379 (28.44 MB)
Trainable params: 7456379 (28.44 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [30]:
# model.fit(X, y, batch_size = 512, epochs = 10)

In [30]:
model.fit(X, y, batch_size = 512, epochs = 20)

Epoch 1/20
391/391 [==============================] - 67s 150ms/step - loss: 6.9716 - accuracy: 0.0281
Epoch 2/20
391/391 [==============================] - 25s 63ms/step - loss: 6.6133 - accuracy: 0.0358
Epoch 3/20
391/391 [==============================] - 18s 46ms/step - loss: 6.3875 - accuracy: 0.0531
Epoch 4/20
391/391 [==============================] - 16s 40ms/step - loss: 6.1302 - accuracy: 0.0741
Epoch 5/20
391/391 [==============================] - 15s 39ms/step - loss: 5.9314 - accuracy: 0.0884
Epoch 6/20
391/391 [==============================] - 15s 37ms/step - loss: 5.7719 - accuracy: 0.0968
Epoch 7/20
391/391 [==============================] - 15s 38ms/step - loss: 5.6343 - accuracy: 0.1038
Epoch 8/20
391/391 [==============================] - 15s 38ms/step - loss: 5.5047 - accuracy: 0.1088
Epoch 9/20
391/391 [==============================] - 14s 36ms/step - loss: 5.3814 - accuracy: 0.1136
Epoch 10/20
391/391 [==============================] - 14s 36ms/step - loss: 5.25

In [31]:
seed_text="From fairest creatures we desire increase"
seed_text

'From fairest creatures we desire increase'

In [32]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen=text_seq_length, truncating='pre')
        
        # Use predict instead of predict_classes
        y_predict = model.predict(encoded)
        
        predicted_word_index = np.argmax(y_predict)
        predicted_word = ""
        
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                predicted_word = word
                break

        seed_text += " " + predicted_word

    print(seed_text)


In [33]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)

1/1 [==============================] - 0s 22ms/step
From fairest creatures we desire increase with the town of embassy to me to me i am a neatherds daughter that i admittance in love to make the gloss with all the cap of the people and he raves and i have seen him to be a ravisher a damned man i have shed to the world i have shed to the capitol and make thee quietly furrows my heart besiege add oft as flowrs are not to make thy self so art thou art experiencd thou art assured to th spire and gladly helps the pitchy view the pale cedar galls the woundless vine and


It's accuracy has been increased as we increased the size of epochos from 10 to 20
